**You must complete the cells with ### Your code goes here ###**


## **Transfer Learning**
Transfer learning in Convolutional Neural Networks (CNNs) is a technique where a model developed for a specific task is reused as the starting point for a model on a second, related task. This approach leverages the knowledge gained from the original task to improve the performance and training efficiency of the new task. Here’s how it works and why it’s useful:


**How Transfer Learning Works**

**Pre-trained Model:**

A CNN is first trained on a large dataset for a task such as image classification. Popular pre-trained models include VGG, ResNet, and Inception, which are trained on large datasets like ImageNet.

**Feature Extraction:**

The pre-trained model has already learned useful features from the initial dataset. For transfer learning, we use the weights and architecture of this pre-trained model and repurpose it for a new, related task.

**Fine-tuning:**

The final layers of the pre-trained model are either replaced or appended with new layers that are specific to the new task. Only these new layers, or sometimes the entire network, are retrained on the new dataset.

In [11]:
import numpy as np
from matplotlib import pyplot as plt
import keras
from keras.datasets import cifar10
from keras.layers import Dense, Conv2D, Flatten, Activation, MaxPool2D, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
import time, datetime

**We load the cifar 10 dataset then separate the first 5 classes of data as the first training set.**
**Images with labels less than 5 are put in set 1, and the rest are put in set 2. Data is also normalized by dividing by 255.**

In [12]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
n_examples = 50000


X1_train = []
X1_test = []
X2_train = []
X2_test = []
Y1_train = []
Y1_test = []
Y2_train = []
Y2_test = []

for ix in range(n_examples):
    if y_train[ix] < 5:
        # put data in set 1
        X1_train.append(x_train[ix]/255.0) #Dividing by 255 is for normalization
        Y1_train.append(y_train[ix])
    else:
        # put data in set 2
        X2_train.append(x_train[ix]/255.0)
        Y2_train.append(y_train[ix])

for ix in range(y_test.shape[0]):
    if y_test[ix] < 5:
        # put data in set 1
        X1_test.append(x_test[ix]/255.0)
        Y1_test.append(y_test[ix])
    else:
        # put data in set 2
        X2_test.append(x_test[ix]/255.0)
        Y2_test.append(y_test[ix])

**Convert the lists to NumPy arrays and reshape them to the correct input format. Labels are one-hot encoded for classification.**

In [13]:
X1_train = np.asarray(X1_train).reshape((-1, 32, 32, 3))
X1_test = np.asarray(X1_test).reshape((-1, 32, 32, 3))
X2_train = np.asarray(X2_train).reshape((-1, 32, 32, 3))
X2_test = np.asarray(X2_test).reshape((-1, 32, 32, 3))

Y1_train = to_categorical(np.asarray(Y1_train), 5)
Y1_test = to_categorical(np.asarray(Y1_test), 5)

Y2_train = to_categorical(np.asarray(Y2_train), 10)
Y2_test = to_categorical(np.asarray(Y2_test), 10)
print (X1_train.shape, X1_test.shape)
print (Y1_train.shape, Y1_test.shape)

(25000, 32, 32, 3) (5000, 32, 32, 3)
(25000, 5) (5000, 5)


**Split the training data into training and validation sets using an 80-20 split.**

In [14]:
split1 = int(0.8 * X1_train.shape[0])
split2 = int(0.8 * X2_train.shape[0])

x1_val = X1_train[split1:]
x1_train = X1_train[:split1]
y1_val = Y1_train[split1:]
y1_train = Y1_train[:split1]

x2_val = X2_train[split2:]
x2_train = X2_train[:split2]
y2_val = Y2_train[split2:]
y2_train = Y2_train[:split2]

**Build and compile Keras CNN model for classification.**

Your layers should be arranged like this:

Layer 1. Convolutional layer with 32 4x4 kernels, expecting input of shape (32, 32, 3) and using relu as activation function.

Layer 2. 2DMaxPooling layer  which pool_size is 2x2.

Layer 3. Convolutional layer with 32 4x4 kernels, expecting input of shape (32, 32, 3) and using relu as activation function.

Layer 4. 2DMaxPooling layer  which pool_size is 2x2.

Layer 5. Flatten layer


Layer 7. Fully connected layer with 256 neurons, using relu as activation function.

Layer 8. Fully connected layer with 5 neurons - acting as the final output classes.

In [17]:
from keras.layers import MaxPooling2D
model = Sequential()
model.add(Conv2D(32, kernel_size=(4, 4), input_shape=(32, 32, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=(4, 4), input_shape=(32, 32, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

**Now it's time to train the network.**

In [18]:
start = datetime.datetime.now()
hist1 = model.fit(x1_train, y1_train,
         epochs=10,
         shuffle=True,
         batch_size=100,
         validation_data=(x1_val, y1_val), verbose=2)

time_taken = datetime.datetime.now() - start
print ('\n'*2, '-'*20, '\n')
print ('Time taken for first training: ', time_taken)
print ('\n', '-'*20, '\n'*2)

Epoch 1/10
200/200 - 21s - 105ms/step - accuracy: 0.5411 - loss: 1.1082 - val_accuracy: 0.6202 - val_loss: 0.9377
Epoch 2/10
200/200 - 22s - 109ms/step - accuracy: 0.6431 - loss: 0.8935 - val_accuracy: 0.6688 - val_loss: 0.8527
Epoch 3/10
200/200 - 42s - 208ms/step - accuracy: 0.6780 - loss: 0.8194 - val_accuracy: 0.6874 - val_loss: 0.8061
Epoch 4/10
200/200 - 39s - 194ms/step - accuracy: 0.7090 - loss: 0.7501 - val_accuracy: 0.7110 - val_loss: 0.7498
Epoch 5/10
200/200 - 22s - 110ms/step - accuracy: 0.7297 - loss: 0.6993 - val_accuracy: 0.7038 - val_loss: 0.7529
Epoch 6/10
200/200 - 41s - 206ms/step - accuracy: 0.7481 - loss: 0.6560 - val_accuracy: 0.7250 - val_loss: 0.7205
Epoch 7/10
200/200 - 20s - 101ms/step - accuracy: 0.7589 - loss: 0.6311 - val_accuracy: 0.7166 - val_loss: 0.7578
Epoch 8/10
200/200 - 22s - 109ms/step - accuracy: 0.7844 - loss: 0.5715 - val_accuracy: 0.7488 - val_loss: 0.6856
Epoch 9/10
200/200 - 20s - 98ms/step - accuracy: 0.7976 - loss: 0.5365 - val_accuracy: 0

**The convolutional neural network for training a model for classes 5..9 by using the information gained by previous model is as follows:**


**Build and compile Follwed Keras CNN model for classification.**

Your layers should be arranged like this:

Layer 1-5: First 5 layers of the previous model (basically the convoluted layers) are used as it is in this model. Further they are made non-trainable to preserve the information gained by them while training of previous model. (So that means sense to freeze them.)

Layer 7: Fully connected layer with 128 neurons, using relu as activation function.

Layer 8. Fully connected layer with 10 neurons - acting as the final output classes.

In [19]:
previous_model = model
new_model = Sequential()
for layer in previous_model.layers[:5]:
    layer.trainable = False
    new_model.add(layer)

new_model.add(Dense(128, activation='relu'))
new_model.add(Dense(10, activation='softmax'))
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

**Now it's time to train the 2nd Part**

In [20]:
trans_model = new_model
start = datetime.datetime.now()
hist2 = trans_model.fit(x2_train, y2_train, epochs=10, shuffle=True, batch_size=100, validation_data=(x2_val, y2_val), verbose=2)
time_taken = datetime.datetime.now() - start
print ('\n'*2, '-'*20, '\n')
print ('Time taken for final training: ', time_taken)
print ('\n', '-'*20, '\n'*2)

Epoch 1/10
200/200 - 8s - 38ms/step - accuracy: 0.7272 - loss: 0.7558 - val_accuracy: 0.7824 - val_loss: 0.5955
Epoch 2/10
200/200 - 9s - 46ms/step - accuracy: 0.7915 - loss: 0.5699 - val_accuracy: 0.7904 - val_loss: 0.5783
Epoch 3/10
200/200 - 11s - 53ms/step - accuracy: 0.8077 - loss: 0.5234 - val_accuracy: 0.7978 - val_loss: 0.5491
Epoch 4/10
200/200 - 11s - 55ms/step - accuracy: 0.8163 - loss: 0.5002 - val_accuracy: 0.8062 - val_loss: 0.5349
Epoch 5/10
200/200 - 11s - 53ms/step - accuracy: 0.8262 - loss: 0.4737 - val_accuracy: 0.8122 - val_loss: 0.5178
Epoch 6/10
200/200 - 6s - 28ms/step - accuracy: 0.8348 - loss: 0.4523 - val_accuracy: 0.8176 - val_loss: 0.5025
Epoch 7/10
200/200 - 10s - 51ms/step - accuracy: 0.8417 - loss: 0.4358 - val_accuracy: 0.8176 - val_loss: 0.5050
Epoch 8/10
200/200 - 11s - 56ms/step - accuracy: 0.8469 - loss: 0.4178 - val_accuracy: 0.8252 - val_loss: 0.4984
Epoch 9/10
200/200 - 11s - 56ms/step - accuracy: 0.8577 - loss: 0.3963 - val_accuracy: 0.8264 - val